
... ***CURRENTLY UNDER DEVELOPMENT*** ...


## Obtain wave families form historical wave partitions
### In this case, two swell systems, one local sea

inputs required: 
  * Historical wave conditions (CSIRO hindcast)
  * Wave families sectors; the split is based on wave direction
  
in this notebook:
  * Split energy based on defined sectors  
  * 
  <span style='color:Red'> TODO: Remove TC associated waves to avoid double counting  </span>

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op
from datetime import datetime

# pip
import numpy as np
import xarray as xr

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..'))

# teslakit
from teslakit.database import Database
from teslakit.waves import GetDistribution



## Database and Site parameters

In [2]:
# --------------------------------------
# Teslakit database

p_data = r'/Users/albacid/Projects/TeslaKit_projects'
db = Database(p_data)

# set site
db.SetSite('KWAJALEIN')



In [5]:
#--------------------------------------
# Load wave families offshore

WVS = xr.open_dataset('/Users/albacid/Projects/TeslaKit_projects/inputs_Kwajalein/waves_families.nc')

print(WVS)
print()
print(WVS.time.isel(time=slice(0,3)))

<xarray.Dataset>
Dimensions:      (time: 111040)
Coordinates:
  * time         (time) datetime64[ns] 1979-01-01 ... 2016-12-31T21:00:00
Data variables:
    swell_2_Hs   (time) float64 ...
    swell_2_Tp   (time) float64 ...
    swell_2_Dir  (time) float64 ...
    swell_1_Hs   (time) float64 ...
    swell_1_Tp   (time) float64 ...
    swell_1_Dir  (time) float64 ...
    sea_Hs       (time) float64 ...
    sea_Tp       (time) float64 ...
    sea_Dir      (time) float64 ...

<xarray.DataArray 'time' (time: 3)>
array(['1979-01-01T00:00:00.000000000', '1979-01-01T03:00:00.000000000',
       '1979-01-01T06:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01 ... 1979-01-01T06:00:00


In [4]:
sys.exit()
'-----------------------------------------------'

SystemExit: 

In [ ]:
# --------------------------------------
# load data and set parameters

WVS_pts = db.Load_WAVES_partitions()  # waves partitions data

_, TCs_r1_params = db.Load_TCs_r1_hist()   # TCs historical parameters inside big radius
_, TCs_r2_params = db.Load_TCs_r2_hist()   # TCs historical parameters inside small radius

# wave families sectors
fams_sectors = [(210, 22.5), (22.5, 135)]

# date limits for TCs removal from waves data, and TC time window (hours)
tc_rm_date1 = '1979-01-01'
tc_rm_date2 = '2015-12-31'
tc_time_window = 12



## Calculate Historical Waves Families

In [ ]:
# --------------------------------------
# Calculate wave families from waves partitions data and waves sectors

WVS = GetDistribution(WVS_pts, fams_sectors)
print(WVS)



## TCs: Waves  Selection 

In [ ]:
# --------------------------------------
# Locate TCs and set category alongside WAVES data  

# remove TCs before 1979 and after 2015 (r1)
dds = TCs_r1_params.dmin_date.values[:]
ix = np.where((dds >= np.datetime64(tc_rm_date1)) & (dds <= np.datetime64(tc_rm_date2)))[0]
TCs_r1_params = TCs_r1_params.isel(storm=ix)

# select storms inside big circle 
storms_sel = TCs_r1_params.storm.values[:]

# add TCs category alongside WAVES data
WVS['TC_category'] = (('time',), np.empty(len(WVS.time))*np.nan)

for s in storms_sel:

    # waves at storm dates
    ss = TCs_r1_params.sel(storm=s)     
    wvs_s = WVS.sel(time = slice(ss.dmin_date, ss.last_date))
    
    # get hs_max date 
    t_hs_max = wvs_s.where(wvs_s.Hs ==  wvs_s.Hs.max(), drop=True).time.values[:][0]
        
    # hs_max time window 
    w1 = t_hs_max - np.timedelta64(tc_time_window,'h')
    w2 = t_hs_max + np.timedelta64(tc_time_window,'h')
    
    # set category alongside WAVES data
    ixs = np.where((WVS.time >= w1) & (WVS.time <= w2))[0]
    WVS['TC_category'][ixs] = ss.category

print(WVS)
    
# Store historical WAVES data
db.Save_WAVES_hist(WVS)
